In [13]:
import hopsworks
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import (
    roc_auc_score,
    precision_recall_fscore_support,
    confusion_matrix,
    mean_absolute_error,
)



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


PROJECT_NAME = os.getenv("HOPSWORKS_PROJECT", "Flight_Predictor_JFK")
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

project = hopsworks.login(
    project=PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

print(f"Connected to Hopsworks project: {project.name}")

2026-01-06 16:12:42,799 INFO: Initializing external client
2026-01-06 16:12:42,799 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-06 16:12:44,291 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1338517
Connected to Hopsworks project: Flight_Predictor_JFK


In [3]:

fs = project.get_feature_store()
fv = fs.get_feature_view("jfk_delay_weather_fv", version=1)


# Pull batch data from Feature View
df = fv.get_batch_data()
print("FV batch shape:", df.shape)

# --- Target engineering ---
# dep_delay can be negative (early departures). We predict minutes late only.
df["delay_min"] = df["dep_delay"].clip(lower=0)

# Optional: cap extreme tail for stability (you can adjust/remove later)
CAP_MINUTES = 240
df["delay_min_capped"] = df["delay_min"].clip(upper=CAP_MINUTES)

# Basic sanity checks
print("\nTarget summary (delay_min):")
print(df["delay_min"].describe())

print("\nCapped target summary (delay_min_capped):")
print(df["delay_min_capped"].describe())

# Drop rows missing target or key fields (keep it minimal for now)
df = df.dropna(subset=["sched_dep_local", "delay_min_capped"])
print("\nAfter dropping missing target/time:", df.shape)

# Ensure sched_dep_local is datetime
df["sched_dep_local"] = pd.to_datetime(df["sched_dep_local"], errors="coerce", utc=True)
df = df.dropna(subset=["sched_dep_local"])
print("After ensuring sched_dep_local datetime:", df.shape)

df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.34s) 
FV batch shape: (107589, 16)

Target summary (delay_min):
count    107589.000000
mean         15.286721
std          50.368883
min           0.000000
25%           0.000000
50%           0.000000
75%           6.000000
max        1656.000000
Name: delay_min, dtype: float64

Capped target summary (delay_min_capped):
count    107589.000000
mean         14.000204
std          37.371524
min           0.000000
25%           0.000000
50%           0.000000
75%           6.000000
max         240.000000
Name: delay_min_capped, dtype: float64

After dropping missing target/time: (107589, 18)
After ensuring sched_dep_local datetime: (107589, 18)


,flight_id,sched_dep_local,sched_hour_local,month,day_of_week,reporting_airline,dest,distance,dep_delay,weather_jfk_hourly_fg_weather_code,weather_jfk_hourly_fg_wind_speed_ms,weather_jfk_hourly_fg_wind_gust_ms,weather_jfk_hourly_fg_temp_c,weather_jfk_hourly_fg_precip_mm,weather_jfk_hourly_fg_snowfall_cm,weather_jfk_hourly_fg_visibility_m,delay_min,delay_min_capped
0,JFK_PBI_B6_2024-10-01 14:00:00-04:00,2024-10-01 18:00:00+00:00,2024-10-01 18:00:00+00:00,10,2,B6,PBI,1028.0,3.0,3,17.8,33.5,21.0,0.0,0.0,NaN,3.0,3.0
1,JFK_PIT_YX_2024-10-01 14:00:00-04:00,2024-10-01 18:00:00+00:00,2024-10-01 18:00:00+00:00,10,2,YX,PIT,340.0,-7.0,3,17.8,33.5,21.0,0.0,0.0,NaN,0.0,0.0
2,JFK_BOS_B6_2024-10-01 14:00:00-04:00,2024-10-01 18:00:00+00:00,2024-10-01 18:00:00+00:00,10,2,B6,BOS,187.0,49.0,3,17.8,33.5,21.0,0.0,0.0,NaN,49.0,49.0
3,JFK_BOS_YX_2024-10-01 14:20:00-04:00,2024-10-01 18:20:00+00:00,2024-10-01 18:00:00+00:00,10,2,YX,BOS,187.0,-3.0,3,17.8,33.5,21.0,0.0,0.0,NaN,0.0,0.0
4,JFK_PIT_9E_2024-10-01 14:25:00-04:00,2024-10-01 18:25:00+00:00,2024-10-01 18:00:00+00:00,10,2,9E,PIT,340.0,-5.0,3,17.8,33.5,21.0,0.0,0.0,NaN,0.0,0.0


In [4]:
df = df.sort_values("sched_dep_local").reset_index(drop=True)

# Choose split boundaries (simple 80/10/10)
n = len(df)
train_end = int(n * 0.80)
val_end   = int(n * 0.90)

train_df = df.iloc[:train_end].copy()
val_df   = df.iloc[train_end:val_end].copy()
test_df  = df.iloc[val_end:].copy()

print("Train/Val/Test sizes:", train_df.shape, val_df.shape, test_df.shape)
print("Train time range:", train_df["sched_dep_local"].min(), "->", train_df["sched_dep_local"].max())
print("Val time range:  ", val_df["sched_dep_local"].min(), "->", val_df["sched_dep_local"].max())
print("Test time range: ", test_df["sched_dep_local"].min(), "->", test_df["sched_dep_local"].max())

Train/Val/Test sizes: (86071, 18) (10759, 18) (10759, 18)
Train time range: 2024-10-01 10:00:00+00:00 -> 2025-07-17 21:47:00+00:00
Val time range:   2025-07-17 21:55:00+00:00 -> 2025-08-24 11:05:00+00:00
Test time range:  2025-08-24 11:20:00+00:00 -> 2025-10-01 02:55:00+00:00


In [5]:
# =========================
# STEP 3: Define features + target
# =========================

target_col = "delay_min_capped"

feature_cols = [
    "month",
    "day_of_week",
    "reporting_airline",
    "dest",
    "distance",
    "weather_jfk_hourly_fg_weather_code",
    "weather_jfk_hourly_fg_wind_speed_ms",
    "weather_jfk_hourly_fg_wind_gust_ms",
    "weather_jfk_hourly_fg_temp_c",
    "weather_jfk_hourly_fg_precip_mm",
    "weather_jfk_hourly_fg_snowfall_cm",
    "weather_jfk_hourly_fg_visibility_m",
]

X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()

X_val = val_df[feature_cols].copy()
y_val = val_df[target_col].copy()

X_test = test_df[feature_cols].copy()
y_test = test_df[target_col].copy()

# Ensure categoricals are strings (important for OneHotEncoder)
for c in ["reporting_airline", "dest"]:
    X_train[c] = X_train[c].astype(str)
    X_val[c] = X_val[c].astype(str)
    X_test[c] = X_test[c].astype(str)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)


X_train: (86071, 12) y_train: (86071,)
X_val: (10759, 12) y_val: (10759,)
X_test: (10759, 12) y_test: (10759,)


In [6]:
cat_cols = ["reporting_airline", "dest"]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median"))
        ]), num_cols),
    ],
    remainder="drop"
)

model = HistGradientBoostingRegressor(
    loss="absolute_error",   # optimizes MAE directly
    max_depth=None,
    learning_rate=0.08,
    max_iter=300,
    random_state=42
)

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", model)
])

pipe.fit(X_train, y_train)

# Validate
val_pred = pipe.predict(X_val)
val_mae = mean_absolute_error(y_val, val_pred)
print("Validation MAE:", val_mae)

# Test
test_pred = pipe.predict(X_test)
test_mae = mean_absolute_error(y_test, test_pred)
print("Test MAE:", test_mae)

2026-01-06 16:13:01,375 WARNING: UserWarning: Skipping features without any observed values: ['weather_jfk_hourly_fg_visibility_m']. At least one non-missing value is needed for imputation with strategy='median'.

2026-01-06 16:13:01,565 WARNING: UserWarning: Skipping features without any observed values: ['weather_jfk_hourly_fg_visibility_m']. At least one non-missing value is needed for imputation with strategy='median'.

Validation MAE: 19.700622734454875
2026-01-06 16:13:01,573 WARNING: UserWarning: Skipping features without any observed values: ['weather_jfk_hourly_fg_visibility_m']. At least one non-missing value is needed for imputation with strategy='median'.

Test MAE: 10.8295380611581


In [7]:
def split_stats(name, y):
    y = y.astype(float)
    delayed = (y > 0)
    print(f"\n{name} target stats:")
    print("Rows:", len(y))
    print("Mean:", y.mean())
    print("Median:", y.median())
    print("75%:", y.quantile(0.75))
    print("95%:", y.quantile(0.95))
    print("Delayed rate (y>0):", delayed.mean())
    if delayed.any():
        print("Mean delay (delayed only):", y[delayed].mean())

split_stats("Train", y_train)
split_stats("Val", y_val)
split_stats("Test", y_test)


Train target stats:
Rows: 86071
Mean: 13.683981828955165
Median: 0.0
75%: 6.0
95%: 84.0
Delayed rate (y>0): 0.3251036934623741
Mean delay (delayed only): 42.09113001215067

Val target stats:
Rows: 10759
Mean: 19.700622734454875
Median: 0.0
75%: 15.0
95%: 117.0
Delayed rate (y>0): 0.39548285156613067
Mean delay (delayed only): 49.81410105757932

Test target stats:
Rows: 10759
Mean: 10.8295380611581
Median: 0.0
75%: 1.0
95%: 69.0
Delayed rate (y>0): 0.2579235988474765
Mean delay (delayed only): 41.987387387387386


In [8]:
val_mae_0 = mean_absolute_error(y_val, np.zeros_like(y_val))
test_mae_0 = mean_absolute_error(y_test, np.zeros_like(y_test))

train_median = float(np.median(y_train))
val_mae_med = mean_absolute_error(y_val, np.full_like(y_val, train_median))
test_mae_med = mean_absolute_error(y_test, np.full_like(y_test, train_median))

print("Baseline (predict 0)   - Val MAE:", val_mae_0, "Test MAE:", test_mae_0)
print("Baseline (train median)- Val MAE:", val_mae_med, "Test MAE:", test_mae_med)
print("Train median:", train_median)

Baseline (predict 0)   - Val MAE: 19.700622734454875 Test MAE: 10.8295380611581
Baseline (train median)- Val MAE: 19.700622734454875 Test MAE: 10.8295380611581
Train median: 0.0


In [9]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Predictions
val_pred = pipe.predict(X_val)
test_pred = pipe.predict(X_test)

# Baseline 0 predictions
val_pred0 = np.zeros_like(y_val, dtype=float)
test_pred0 = np.zeros_like(y_test, dtype=float)

print("VAL  - Model MAE:", mean_absolute_error(y_val, val_pred),
      "| Baseline0 MAE:", mean_absolute_error(y_val, val_pred0))

print("TEST - Model MAE:", mean_absolute_error(y_test, test_pred),
      "| Baseline0 MAE:", mean_absolute_error(y_test, test_pred0))

2026-01-06 16:13:01,605 WARNING: UserWarning: Skipping features without any observed values: ['weather_jfk_hourly_fg_visibility_m']. At least one non-missing value is needed for imputation with strategy='median'.

2026-01-06 16:13:01,611 WARNING: UserWarning: Skipping features without any observed values: ['weather_jfk_hourly_fg_visibility_m']. At least one non-missing value is needed for imputation with strategy='median'.

VAL  - Model MAE: 19.700622734454875 | Baseline0 MAE: 19.700622734454875
TEST - Model MAE: 10.8295380611581 | Baseline0 MAE: 10.8295380611581


In [10]:
val_mask = y_val > 0
test_mask = y_test > 0

print("VAL delayed-only MAE:",
      mean_absolute_error(y_val[val_mask], val_pred[val_mask]),
      "| Baseline0:",
      mean_absolute_error(y_val[val_mask], val_pred0[val_mask]))

print("TEST delayed-only MAE:",
      mean_absolute_error(y_test[test_mask], test_pred[test_mask]),
      "| Baseline0:",
      mean_absolute_error(y_test[test_mask], test_pred0[test_mask]))

VAL delayed-only MAE: 49.81410105757932 | Baseline0: 49.81410105757932
TEST delayed-only MAE: 41.987387387387386 | Baseline0: 41.987387387387386


In [11]:
from xgboost import XGBRegressor
import numpy as np

# ---- Drop visibility (all missing) ----
feature_cols_xgb = [c for c in feature_cols if c != "weather_jfk_hourly_fg_visibility_m"]

cat_cols = ["reporting_airline", "dest"]
num_cols = [c for c in feature_cols_xgb if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),  # sparse OK
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median"))
        ]), num_cols),
    ],
    remainder="drop"
)

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

pipe_xgb = Pipeline(steps=[
    ("prep", preprocess),
    ("model", xgb)
])

pipe_xgb.fit(X_train[feature_cols_xgb], y_train)

# Predictions
val_pred = pipe_xgb.predict(X_val[feature_cols_xgb])
test_pred = pipe_xgb.predict(X_test[feature_cols_xgb])

# Baseline
val_pred0 = np.zeros_like(y_val, dtype=float)
test_pred0 = np.zeros_like(y_test, dtype=float)

print("VAL  - XGB MAE:", mean_absolute_error(y_val, val_pred),
      "| Baseline0 MAE:", mean_absolute_error(y_val, val_pred0))

print("TEST - XGB MAE:", mean_absolute_error(y_test, test_pred),
      "| Baseline0 MAE:", mean_absolute_error(y_test, test_pred0))

# Delayed-only evaluation
val_mask = y_val > 0
test_mask = y_test > 0

print("VAL delayed-only MAE:",
      mean_absolute_error(y_val[val_mask], val_pred[val_mask]),
      "| Baseline0:",
      mean_absolute_error(y_val[val_mask], val_pred0[val_mask]))

print("TEST delayed-only MAE:",
      mean_absolute_error(y_test[test_mask], test_pred[test_mask]),
      "| Baseline0:",
      mean_absolute_error(y_test[test_mask], test_pred0[test_mask]))

VAL  - XGB MAE: 30.189896919915977 | Baseline0 MAE: 19.700622734454875
TEST - XGB MAE: 24.363262792083702 | Baseline0 MAE: 10.8295380611581
VAL delayed-only MAE: 40.434668242346525 | Baseline0: 49.81410105757932
TEST delayed-only MAE: 34.687796006267135 | Baseline0: 41.987387387387386


In [14]:
# ----------------------------
# 0) Labels
# ----------------------------
# Assumes you already have: train_df, val_df, test_df and delay_min_capped in them

for _df in [train_df, val_df, test_df]:
    _df["is_delayed"] = (_df["delay_min_capped"] > 0).astype(int)

In [15]:
# ----------------------------
# 1) Feature columns
# ----------------------------
feature_cols = [
    "month",
    "day_of_week",
    "reporting_airline",
    "dest",
    "distance",
    "weather_jfk_hourly_fg_weather_code",
    "weather_jfk_hourly_fg_wind_speed_ms",
    "weather_jfk_hourly_fg_wind_gust_ms",
    "weather_jfk_hourly_fg_temp_c",
    "weather_jfk_hourly_fg_precip_mm",
    "weather_jfk_hourly_fg_snowfall_cm",
    # drop visibility (all missing in your data)
    # "weather_jfk_hourly_fg_visibility_m",
]

cat_cols = ["reporting_airline", "dest"]
num_cols = [c for c in feature_cols if c not in cat_cols]

def make_Xy(_df):
    X = _df[feature_cols].copy()
    X["reporting_airline"] = X["reporting_airline"].astype(str)
    X["dest"] = X["dest"].astype(str)
    y_cls = _df["is_delayed"].astype(int).copy()
    y_reg = _df["delay_min_capped"].astype(float).copy()
    return X, y_cls, y_reg

X_train, y_train_cls, y_train_reg = make_Xy(train_df)
X_val,   y_val_cls,   y_val_reg   = make_Xy(val_df)
X_test,  y_test_cls,  y_test_reg  = make_Xy(test_df)

print("Train delayed rate:", y_train_cls.mean())
print("Val delayed rate:  ", y_val_cls.mean())
print("Test delayed rate: ", y_test_cls.mean())

Train delayed rate: 0.3251036934623741
Val delayed rate:   0.39548285156613067
Test delayed rate:  0.2579235988474765


In [16]:
# ----------------------------
# 2) Shared preprocessing
# ----------------------------
# Use sparse one-hot (fine for LogisticRegression). Numeric: median impute.
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

In [17]:
# ============================================================
# STAGE 1: CLASSIFIER
# ============================================================

clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",   # helps imbalance
    n_jobs=-1
)

clf_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", clf)
])

clf_pipe.fit(X_train, y_train_cls)

# Probabilities
val_proba = clf_pipe.predict_proba(X_val)[:, 1]
test_proba = clf_pipe.predict_proba(X_test)[:, 1]

# AUC
val_auc = roc_auc_score(y_val_cls, val_proba)
test_auc = roc_auc_score(y_test_cls, test_proba)
print("\n[Stage 1] ROC AUC - Val:", val_auc, "Test:", test_auc)

# ---- Pick threshold by maximizing F1 on validation ----
thresholds = np.linspace(0.05, 0.95, 19)
best = {"thr": None, "f1": -1, "precision": None, "recall": None}

for thr in thresholds:
    val_pred_cls = (val_proba >= thr).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_val_cls, val_pred_cls, average="binary", zero_division=0)
    if f1 > best["f1"]:
        best = {"thr": float(thr), "f1": float(f1), "precision": float(p), "recall": float(r)}

print("[Stage 1] Best threshold on Val (max F1):", best)

# Confusion matrix at best threshold (val)
val_pred_cls_best = (val_proba >= best["thr"]).astype(int)
cm_val = confusion_matrix(y_val_cls, val_pred_cls_best)
print("\n[Stage 1] Val confusion matrix @ thr=", best["thr"])
print(cm_val)


2026-01-06 16:25:27,570 WARNING: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.

2026-01-06 16:25:31,124 WARNING: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[Stage 1] ROC AUC - Val: 0.6600738874603788 Test: 0.6054330056509415
[Stage 1] Best threshold on Val (max F1): {'thr': 0.44999999999999996, 'f1': 0.5901193049883898, 'precision': 0.4475346125819772, 'recall': 0.8660399529964747}

[Stage 1] Val confusion matrix @ thr= 0.44999999999999996
[[1955 4549]
 [ 570 3685]]


In [21]:
# ============================================================
# STAGE 2: REGRESSOR (train only on delayed flights)
# ============================================================

train_delayed_mask = (y_train_cls == 1)
val_delayed_mask   = (y_val_cls == 1)
test_delayed_mask  = (y_test_cls == 1)

X_train_del = X_train.loc[train_delayed_mask].copy()
y_train_del = y_train_reg.loc[train_delayed_mask].copy()

X_val_del = X_val.loc[val_delayed_mask].copy()
y_val_del = y_val_reg.loc[val_delayed_mask].copy()

X_test_del = X_test.loc[test_delayed_mask].copy()
y_test_del = y_test_reg.loc[test_delayed_mask].copy()

print("\nDelayed-only rows - Train:", X_train_del.shape, "Val:", X_val_del.shape, "Test:", X_test_del.shape)

# Use a regressor that accepts dense output -> set OneHotEncoder dense for regressor pipeline
preprocess_dense = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

reg = HistGradientBoostingRegressor(
    loss="squared_error",   # avoids collapsing to 0 like MAE loss did
    learning_rate=0.05,
    max_iter=600,
    random_state=42
)

reg_pipe = Pipeline(steps=[
    ("prep", preprocess_dense),
    ("reg", reg)
])

reg_pipe.fit(X_train_del, y_train_del)

# Delayed-only MAE
val_pred_del = reg_pipe.predict(X_val_del)
test_pred_del = reg_pipe.predict(X_test_del)

print("\n[Stage 2] Delayed-only MAE - Val:", mean_absolute_error(y_val_del, val_pred_del))
print("[Stage 2] Delayed-only MAE - Test:", mean_absolute_error(y_test_del, test_pred_del))


Delayed-only rows - Train: (27982, 11) Val: (4255, 11) Test: (2775, 11)

[Stage 2] Delayed-only MAE - Val: 45.90935129497509
[Stage 2] Delayed-only MAE - Test: 37.74757546710937


In [18]:
# ============================================================
# STAGE 2: REGRESSOR (XGBoost) trained only on delayed flights
# ============================================================

train_delayed_mask = (y_train_cls == 1)
val_delayed_mask   = (y_val_cls == 1)
test_delayed_mask  = (y_test_cls == 1)

X_train_del = X_train.loc[train_delayed_mask].copy()
y_train_del = y_train_reg.loc[train_delayed_mask].copy()

X_val_del = X_val.loc[val_delayed_mask].copy()
y_val_del = y_val_reg.loc[val_delayed_mask].copy()

X_test_del = X_test.loc[test_delayed_mask].copy()
y_test_del = y_test_reg.loc[test_delayed_mask].copy()

print("Delayed-only rows - Train:", X_train_del.shape, "Val:", X_val_del.shape, "Test:", X_test_del.shape)

# IMPORTANT: XGBoost works well with sparse one-hot.
# We'll reuse the SAME preprocess (sparse) used in Stage 1.
# So: transform X with preprocess, then fit XGB on the transformed matrices.

# Fit preprocess on FULL train (so consistent feature space), then transform splits
X_train_mat = preprocess.fit_transform(X_train)
X_val_mat   = preprocess.transform(X_val)
X_test_mat  = preprocess.transform(X_test)

# Delayed-only matrices
X_train_del_mat = X_train_mat[train_delayed_mask.values]
X_val_del_mat   = X_val_mat[val_delayed_mask.values]
X_test_del_mat  = X_test_mat[test_delayed_mask.values]

xgb_reg = XGBRegressor(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

xgb_reg.fit(X_train_del_mat, y_train_del)

# Delayed-only MAE
val_pred_del = xgb_reg.predict(X_val_del_mat)
test_pred_del = xgb_reg.predict(X_test_del_mat)

print("\n[Stage 2 - XGB] Delayed-only MAE - Val:", mean_absolute_error(y_val_del, val_pred_del))
print("[Stage 2 - XGB] Delayed-only MAE - Test:", mean_absolute_error(y_test_del, test_pred_del))


Delayed-only rows - Train: (27982, 11) Val: (4255, 11) Test: (2775, 11)

[Stage 2 - XGB] Delayed-only MAE - Val: 46.92504991465254
[Stage 2 - XGB] Delayed-only MAE - Test: 44.23751353564563


In [19]:
# ============================================================
# COMBINED PREDICTOR: classifier gate + xgb_reg magnitude
# ============================================================

def two_stage_predict_xgb(X_mat, proba, thr):
    """
    X_mat: preprocessed matrix for X (same rows)
    If proba < thr -> predict 0
    else -> predict XGB regressor(X)
    """
    yhat = np.zeros(len(proba), dtype=float)
    delayed_idx = proba >= thr
    if delayed_idx.any():
        yhat[delayed_idx] = xgb_reg.predict(X_mat[delayed_idx])
    return yhat

val_pred_final = two_stage_predict_xgb(X_val_mat, val_proba, best["thr"])
test_pred_final = two_stage_predict_xgb(X_test_mat, test_proba, best["thr"])

print("\n[Combined] Overall MAE - Val:", mean_absolute_error(y_val_reg, val_pred_final))
print("[Combined] Overall MAE - Test:", mean_absolute_error(y_test_reg, test_pred_final))

print("\n[Combined] Delayed-only MAE - Val:",
      mean_absolute_error(y_val_reg[val_delayed_mask], val_pred_final[val_delayed_mask]))
print("[Combined] Delayed-only MAE - Test:",
      mean_absolute_error(y_test_reg[test_delayed_mask], test_pred_final[test_delayed_mask]))


[Combined] Overall MAE - Val: 40.88596477698234
[Combined] Overall MAE - Test: 37.73412207788623

[Combined] Delayed-only MAE - Val: 46.84759210335522
[Combined] Delayed-only MAE - Test: 43.88036187386727


In [20]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# We already have:
# - val_proba, test_proba
# - X_val_mat, X_test_mat (preprocessed matrices)
# - y_val_reg, y_test_reg (true capped delays)
# - xgb_reg (stage2 regressor)

def combined_pred(proba, X_mat, thr):
    yhat = np.zeros(len(proba), dtype=float)
    idx = proba >= thr
    if idx.any():
        yhat[idx] = xgb_reg.predict(X_mat[idx])
    return yhat

thresholds = np.linspace(0.0, 0.95, 20)  # coarse grid first
results = []

for thr in thresholds:
    val_pred = combined_pred(val_proba, X_val_mat, thr)
    mae = mean_absolute_error(y_val_reg, val_pred)
    results.append((thr, mae))

best_thr, best_mae = min(results, key=lambda x: x[1])

print("Threshold tuning (Val overall MAE):")
for thr, mae in results:
    print(f"thr={thr:.2f}  val_MAE={mae:.4f}")

print("\n✅ Best threshold:", best_thr, "with Val MAE:", best_mae)

# Evaluate on Test with best threshold
test_pred = combined_pred(test_proba, X_test_mat, best_thr)
print("TEST overall MAE @ best_thr:", mean_absolute_error(y_test_reg, test_pred))

Threshold tuning (Val overall MAE):
thr=0.00  val_MAE=50.9446
thr=0.05  val_MAE=50.9446
thr=0.10  val_MAE=50.9446
thr=0.15  val_MAE=50.9446
thr=0.20  val_MAE=50.9446
thr=0.25  val_MAE=50.9157
thr=0.30  val_MAE=50.5549
thr=0.35  val_MAE=48.5024
thr=0.40  val_MAE=44.9644
thr=0.45  val_MAE=40.8860
thr=0.50  val_MAE=36.7728
thr=0.55  val_MAE=32.7229
thr=0.60  val_MAE=28.0537
thr=0.65  val_MAE=23.3845
thr=0.70  val_MAE=20.9688
thr=0.75  val_MAE=19.8300
thr=0.80  val_MAE=19.6769
thr=0.85  val_MAE=19.6903
thr=0.90  val_MAE=19.7006
thr=0.95  val_MAE=19.7006

✅ Best threshold: 0.7999999999999999 with Val MAE: 19.676916746814427
TEST overall MAE @ best_thr: 10.93470743453927


In [22]:
import os
import json
import joblib
from datetime import datetime

# -----------------------
# Local artifact paths
# -----------------------
ART_DIR = "model_artifacts"
os.makedirs(ART_DIR, exist_ok=True)

stage1_file = os.path.join(ART_DIR, "stage1_classifier.pkl")
stage2_file = os.path.join(ART_DIR, "stage2_regressor.pkl")
meta_file   = os.path.join(ART_DIR, "two_stage_metadata.json")

# Save the sklearn pipelines
joblib.dump(clf_pipe, stage1_file)
joblib.dump(reg_pipe, stage2_file)

# Save threshold + feature schema info (super important for later inference)
metadata = {
    "created_at": datetime.utcnow().isoformat() + "Z",
    "problem": "JFK departure delay minutes (two-stage)",
    "stage1": {
        "type": "classifier",
        "model": type(clf_pipe[-1]).__name__ if hasattr(clf_pipe, "__getitem__") else str(type(clf_pipe)),
        "cat_cols": cat_cols,
        "num_cols": num_cols,
    },
    "stage2": {
        "type": "regressor",
        "model": type(reg_pipe[-1]).__name__ if hasattr(reg_pipe, "__getitem__") else str(type(reg_pipe)),
    },
    "decision_rule": {
        "threshold_type": "MAE-tuned",
        "best_threshold": float(best_thr),
        "rule": "if P(delay)>threshold => predict regressor minutes else 0"
    },
}

# If you have feature_cols, store it too
if "feature_cols" in globals():
    metadata["feature_cols"] = feature_cols

with open(meta_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("Saved:", stage1_file)
print("Saved:", stage2_file)
print("Saved:", meta_file)

2026-01-06 17:02:22,221 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

Saved: model_artifacts/stage1_classifier.pkl
Saved: model_artifacts/stage2_regressor.pkl
Saved: model_artifacts/two_stage_metadata.json


In [23]:
# Hopsworks Model Registry
mr = project.get_model_registry()

# -----------------------
# Stage 1 registration
# -----------------------
stage1_metrics = {}
# Add whatever you computed
if "val_auc" in globals():  stage1_metrics["val_auc"] = float(val_auc)
if "test_auc" in globals(): stage1_metrics["test_auc"] = float(test_auc)

# (Optional) add overall MAE at best_thr if you computed it
if "val_mae_bestthr" in globals():  stage1_metrics["val_overall_mae_bestthr"] = float(val_mae_bestthr)
if "test_mae_bestthr" in globals(): stage1_metrics["test_overall_mae_bestthr"] = float(test_mae_bestthr)

stage1_model = mr.sklearn.create_model(
    name="jfk_delay_stage1_classifier",
    metrics=stage1_metrics,
    description="Stage 1 classifier for JFK departure delay: predicts P(delay>0)."
)

stage1_model.save(stage1_file)
print("Registered Stage 1:", stage1_model.name, "version:", stage1_model.version)

# -----------------------
# Stage 2 registration
# -----------------------
stage2_metrics = {}
if "stage2_val_mae_del" in globals():  stage2_metrics["val_delayed_only_mae"] = float(stage2_val_mae_del)
if "stage2_test_mae_del" in globals(): stage2_metrics["test_delayed_only_mae"] = float(stage2_test_mae_del)

stage2_model = mr.sklearn.create_model(
    name="jfk_delay_stage2_regressor",
    metrics=stage2_metrics,
    description="Stage 2 regressor for JFK departure delay minutes, trained on delayed flights only."
)

stage2_model.save(stage2_file)
print("Registered Stage 2:", stage2_model.name, "version:", stage2_model.version)

# -----------------------
# Also upload the metadata JSON as a Python model artifact (recommended)
# -----------------------
meta_model = mr.python.create_model(
    name="jfk_delay_two_stage_metadata",
    metrics={"best_threshold": float(best_thr)},
    description="Metadata for two-stage JFK delay system (threshold + feature schema)."
)
meta_model.save(ART_DIR)   # saves the whole folder (includes json + both pkls)
print("Registered metadata bundle:", meta_model.name, "version:", meta_model.version)

2026-01-06 17:03:11,554 WARNING: ProvenanceWarning: Model schema cannot not be inferred without both the feature view and the training dataset version.



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/lakshmisrinidhpachabotla/Desktop/scl_ml_dl/model_artifacts/stage1_classifier.pkl: 0.000%|    …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1338517/models/jfk_delay_stage1_classifier/1
Registered Stage 1: jfk_delay_stage1_classifier version: 1


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/lakshmisrinidhpachabotla/Desktop/scl_ml_dl/model_artifacts/stage1_classifier.pkl: 0.000%|    …

Uploading /Users/lakshmisrinidhpachabotla/Desktop/scl_ml_dl/model_artifacts/two_stage_metadata.json: 0.000%|  …

Uploading /Users/lakshmisrinidhpachabotla/Desktop/scl_ml_dl/model_artifacts/stage2_regressor.pkl: 0.000%|     …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1338517/models/jfk_delay_two_stage_metadata/1
Registered metadata bundle: jfk_delay_two_stage_metadata version: 1


In [24]:
# You already have this file from before
stage2_file = "model_artifacts/stage2_regressor.pkl"

In [26]:
stage2_val_mae_del = mean_absolute_error(y_val_del, val_pred_del)
stage2_test_mae_del = mean_absolute_error(y_test_del, test_pred_del)

print("\n[Stage 2] Delayed-only MAE - Val:", stage2_val_mae_del)
print("[Stage 2] Delayed-only MAE - Test:", stage2_test_mae_del)


[Stage 2] Delayed-only MAE - Val: 45.90935129497509
[Stage 2] Delayed-only MAE - Test: 37.74757546710937


In [27]:
mr = project.get_model_registry()

stage2_metrics = {
    "val_delayed_only_mae": float(stage2_val_mae_del),
    "test_delayed_only_mae": float(stage2_test_mae_del),
}

stage2_model_v2 = mr.sklearn.create_model(
    name="jfk_delay_stage2_regressor",  # SAME NAME -> new version
    metrics=stage2_metrics,
    description=(
        "Stage 2 regressor for JFK delay minutes (trained on delayed flights only). "
        "This version adds delayed-only MAE metrics for val/test."
    ),
)

# IMPORTANT: save the same artifact file again -> creates a new version
stage2_model_v2.save("model_artifacts/stage2_regressor.pkl")

print(f"✅ Created new version: {stage2_model_v2.name} v{stage2_model_v2.version}")

2026-01-06 17:10:51,144 WARNING: ProvenanceWarning: Model schema cannot not be inferred without both the feature view and the training dataset version.



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/lakshmisrinidhpachabotla/Desktop/scl_ml_dl/model_artifacts/stage2_regressor.pkl: 0.000%|     …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1338517/models/jfk_delay_stage2_regressor/2
✅ Created new version: jfk_delay_stage2_regressor v2
